In [72]:
from dotenv import load_dotenv
import pandas as pd
import numpy as np
from polygon import RESTClient
from datetime import datetime,timezone
import pandas_market_calendars as mcal
import polygon
import os

### Data Collection

In [3]:
load_dotenv("/Users/brad/mlprojects/guidelight/guidelight-api/.env")
token = os.getenv("POLYGON_TOKEN")

In [4]:
client = RESTClient(api_key=token)

In [78]:
a = client.list_options_contracts(underlying_ticker="AAPL", expired=False, as_of="2023-04-04", contract_type="put")
option_contracts =  list(a)

In [79]:
print(option_contracts[0].ticker)

list(client.list_trades(option_contracts[0].ticker, timestamp="2023-04-04"))[0].participant_timestamp

O:AAPL230406P00050000


1680626651067000000

In [90]:
colnames = list(option_contracts[0].__dict__.keys())
cols = colnames.copy()
colnames.extend(["maturity"])
print(colnames)
df = pd.DataFrame(columns=colnames)
for contract in option_contracts:
	trades = list(client.list_trades(contract.ticker, timestamp="2023-03-27"))
	if len(trades) > 0:
		last_trade_time = datetime.utcfromtimestamp(trades[0].participant_timestamp / 1000000000)
		expiration_time = datetime.strptime(contract.expiration_date, "%Y-%m-%d")
		if (last_trade_time - expiration_time).days > -14:
			today =  datetime.strptime("2023-03-27", "%Y-%m-%d")
	
			a = pd.DataFrame([contract.__dict__.values()], columns=cols)
			a['maturity'] = (expiration_time - today).days / 365
			df = pd.concat([df, a])

df.to_csv("option_2023-03-27.csv")

['additional_underlyings', 'cfi', 'contract_type', 'correction', 'exercise_style', 'expiration_date', 'primary_exchange', 'shares_per_contract', 'strike_price', 'ticker', 'underlying_ticker', 'maturity']


/var/folders/rg/cmtcr_3n3g5bk2x49f7jdpdw0000gn/T/ipykernel_474/4130836677.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, a])


In [26]:
mcal.get_calendar("NYSE").valid_days(start_date='2023-01-01', end_date='2023-12-31')

DatetimeIndex(['2023-01-03 00:00:00+00:00', '2023-01-04 00:00:00+00:00',
               '2023-01-05 00:00:00+00:00', '2023-01-06 00:00:00+00:00',
               '2023-01-09 00:00:00+00:00', '2023-01-10 00:00:00+00:00',
               '2023-01-11 00:00:00+00:00', '2023-01-12 00:00:00+00:00',
               '2023-01-13 00:00:00+00:00', '2023-01-17 00:00:00+00:00',
               ...
               '2023-12-15 00:00:00+00:00', '2023-12-18 00:00:00+00:00',
               '2023-12-19 00:00:00+00:00', '2023-12-20 00:00:00+00:00',
               '2023-12-21 00:00:00+00:00', '2023-12-22 00:00:00+00:00',
               '2023-12-26 00:00:00+00:00', '2023-12-27 00:00:00+00:00',
               '2023-12-28 00:00:00+00:00', '2023-12-29 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', length=250, freq=None)

In [22]:
df

,cfi,contract_type,exercise_style,expiration_date,primary_exchange,shares_per_contract,strike_price,ticker,underlying_ticker
0,OPASPS,put,american,2023-04-06,BATO,100,50,O:AAPL230406P00050000,AAPL
0,OPASPS,put,american,2023-04-06,BATO,100,55,O:AAPL230406P00055000,AAPL
0,OPASPS,put,american,2023-04-06,BATO,100,60,O:AAPL230406P00060000,AAPL
0,OPASPS,put,american,2023-04-06,BATO,100,65,O:AAPL230406P00065000,AAPL
0,OPASPS,put,american,2023-04-06,BATO,100,70,O:AAPL230406P00070000,AAPL
...,...,...,...,...,...,...,...,...,...
0,OPASPS,put,american,2025-12-19,BATO,100,270,O:AAPL251219P00270000,AAPL
0,OPASPS,put,american,2025-12-19,BATO,100,280,O:AAPL251219P00280000,AAPL
0,OPASPS,put,american,2025-12-19,BATO,100,290,O:AAPL251219P00290000,AAPL
0,OPASPS,put,american,2025-12-19,BATO,100,300,O:AAPL251219P00300000,AAPL


In [15]:
yield_rates = pd.read_csv("five-year-rates.csv")
yield_rates

,Date,1 Mo,2 Mo,3 Mo,4 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
0,04/05/2024,5.47,5.50,5.43,5.41,5.34,5.05,4.73,4.54,4.38,4.39,4.39,4.65,4.54
1,04/04/2024,5.47,5.49,5.41,5.40,5.32,5.00,4.65,4.46,4.30,4.31,4.31,4.57,4.47
2,04/03/2024,5.47,5.44,5.42,5.40,5.33,5.03,4.68,4.48,4.34,4.36,4.36,4.61,4.51
3,04/02/2024,5.49,5.45,5.42,5.40,5.34,5.05,4.70,4.51,4.35,4.37,4.36,4.61,4.51
4,04/01/2024,5.49,5.47,5.44,5.41,5.36,5.06,4.72,4.51,4.34,4.33,4.33,4.58,4.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1062,01/08/2020,1.50,1.53,1.54,NaN,1.56,1.55,1.58,1.61,1.67,1.78,1.87,2.21,2.35
1063,01/07/2020,1.52,1.53,1.54,NaN,1.56,1.53,1.54,1.55,1.62,1.74,1.83,2.16,2.31
1064,01/06/2020,1.54,1.54,1.56,NaN,1.56,1.54,1.54,1.56,1.61,1.72,1.81,2.13,2.28
1065,01/03/2020,1.52,1.55,1.52,NaN,1.55,1.55,1.53,1.54,1.59,1.71,1.80,2.11,2.26


In [ ]:
ticker = "AAPL"

# List Aggregates (Bars)
aggs = []
for a in client.list_aggs(ticker=ticker, multiplier=1, timespan="minute", from_="2022-01-01", to="2023-12-30", limit=50000, options=T):
    aggs.append(a)

print(len(aggs))



In [ ]:
aggs[-1]

In [ ]:
#unix millisecond timestamp to datetime
# from pytz import timezone

# Convert timestamp to datetime in UTC timezone
def ts_to_dt(timestamp):
    return datetime.fromtimestamp(timestamp/1000, tz=timezone.utc)

timestamps = np.vectorize(ts_to_dt)

# Example usage
a = np.array([a.timestamp for a in aggs])
times = timestamps(a)
aapl_aggs = pd.DataFrame({"day": [time.strftime("%Y-%m-%d") for time in times],
                            "timestamp": [time.strftime("%Y-%m-%d, %H:%M") for time in times],
                           "close": [a.close for a in aggs], 
                           "volume": [a.volume for a in aggs],
                           "low": [a.low for a in aggs],
                           "high": [a.high for a in aggs]})



In [ ]:
from heston_param import *

In [ ]:
aapl_aggs.groupby("day")

In [ ]:
daily_aggs =  [aapl_aggs.groupby('day').get_group(x) for x in aapl_aggs['day'].unique()]

In [ ]:
hist_voilatilities = torch.empty(len(daily_aggs))
for i, day in enumerate(daily_aggs):
	hist_voilatilities[i] = estimate_historical_volatility(day['close'].values)

# calculate the historical volatility for each day





In [ ]:
import pickle
from heston_param import *
hist_voilatilities = pickle.load(open("hist_voilatilities.pkl", "rb"))
daily_aggs = pickle.load(open("daily_aggs.pkl", "rb"))

In [ ]:
while True:
	try:
		print(calibrate_daily_parameters(hist_voilatilities[0], 0.1, daily_aggs[0]["close"].values, 0.0237, daily_aggs[0]["close"].values.shape[0], 50))
		break
	except RuntimeError:
		continue

In [ ]:
def single_day_calibration(args):
		i, hist_volatilities, daily_agg = args
		while True:
			try:
				params = calibrate_daily_parameters(hist_volatilities[i], 0.1, day["close"].values, 0.0237, day["close"].values.shape[0], 300)
				
				return i, params
			except RuntimeError:
				continue

In [ ]:
len(daily_aggs)

### Naive Monte Carlo:
Runtime: 1.15 Hour. Suboptimal.

In [ ]:
daily_params = np.empty((len(daily_aggs), 5))
for i, day in enumerate((daily_aggs[:20])):
	while True:
		try:
			daily_params[i] = calibrate_daily_parameters(hist_voilatilities[i], 0.1, daily_aggs[i]["close"].values, 0.0237, daily_aggs[i]["close"].values.shape[0], 50)
			if (i + 1) % 10 == 0:
					rate =  100 *  np.round((i + 1) /len(daily_params[:20]), 2)
					print(f"{rate}% completed.")
			
			break
		except RuntimeError:
			continue



In [ ]:
def calibrate_worker(args):
	single_day_calibration(args)

In [ ]:
daily_params = pickle.load(open("parameters.pkl", "rb"))

In [ ]:
# runtime is 1.5 hours
daily_params.shape

In [ ]:
from multiprocess import Pool

def calibrate_parameters_multiprocessing(daily_aggs, hist_volatilities):
    # Prepare arguments for each task
    tasks = [(i, hist_volatilities[i], daily_aggs[i]) for i in range(len(daily_aggs))]

    # Number of processes, could be set to the number of CPUs or cores
    num_processes = 4

    # Create a multiprocessing pool and map tasks to worker function
    with Pool(processes=num_processes) as pool:
        results = pool.map(calibrate_worker, tasks)

    # Process results
    daily_params = np.empty((len(daily_aggs[:20]), 5))
    for index, params in results:
        if params is not None:
            daily_params[index] = params
        else:
            print(f"Calibration failed for index {index}")

    return daily_params



In [ ]:
calibrate_parameters_multiprocessing(daily_aggs[:20], hist_voilatilities[:20])

https://medium.com/@alexander.tsoskounoglou/pricing-options-with-fourier-series-p3-the-heston-model-d157369a217a

In [ ]:
def heston_char(u, params):
    kappa, theta, zeta, rho, v0, r, q, T, S0 = params 
    t0 = 0.0 ;  q = 0.0
    m = np.log(S0) + (r - q)*(T-t0)
    D = np.sqrt((rho*zeta*1j*u - kappa)**2 + zeta**2*(1j*u + u**2))
    C = (kappa - rho*zeta*1j*u - D) / (kappa - rho*zeta*1j*u + D)
    beta = ((kappa - rho*zeta*1j*u - D)*(1-np.exp(-D*(T-t0)))) / (zeta**2*(1-C*np.exp(-D*(T-t0))))
    alpha = ((kappa*theta)/(zeta**2))*((kappa - rho*zeta*1j*u - D)*(T-t0) - 2*np.log((1-C*np.exp(-D*(T-t0))/(1-C))))
    return np.exp(1j*u*m + alpha + beta*v0)

In [4]:
import numpy as np
from numpy import sqrt, exp, pi, cos, sin, log, abs
from numba import njit

@njit
def Fourier_Heston_Put(S0, K, T, r, 
                  # Heston Model Paramters
                  kappa, # Speed of the mean reversion 
                  theta, # Long term mean
                  rho,   # correlation between 2 random variables
                  zeta,  # Volatility of volatility
                  v0,    # Initial volatility 
                  opt_type,
                  N = 1_012,
                  z = 24
                  ):

  def heston_char(u): 
    t0 = 0.0 ;  q = 0.0
    m = log(S0) + (r - q)*(T-t0)
    D = sqrt((rho*zeta*1j*u - kappa)**2 + zeta**2*(1j*u + u**2))
    C = (kappa - rho*zeta*1j*u - D) / (kappa - rho*zeta*1j*u + D)
    beta = ((kappa - rho*zeta*1j*u - D)*(1-exp(-D*(T-t0)))) / (zeta**2*(1-C*exp(-D*(T-t0))))
    alpha = ((kappa*theta)/(zeta**2))*((kappa - rho*zeta*1j*u - D)*(T-t0) - 2*log((1-C*exp(-D*(T-t0))/(1-C))))
    return exp(1j*u*m + alpha + beta*v0)
  
  # # Parameters for the Function to make sure the approximations are correct.
  c1 = log(S0) + r*T - .5*theta*T
  c2 = theta/(8*kappa**3)*(-zeta**2*exp(-2*kappa*T) + 4*zeta*exp(-kappa*T)*(zeta-2*kappa*rho) 
        + 2*kappa*T*(4*kappa**2 + zeta**2 - 4*kappa*zeta*rho) + zeta*(8*kappa*rho - 3*zeta))
  a = c1 - z*sqrt(abs(c2))
  b = c1 + z*sqrt(abs(c2))
  
  h       = lambda n : (n*pi) / (b-a) 
  g_n     = lambda n : (exp(a) - (K/h(n))*sin(h(n)*(a - log(K))) - K*cos(h(n)*(a - log(K)))) / (1 + h(n)**2)
  g0      = K*(log(K) - a - 1) + exp(a)
  
  F = g0 
  for n in range(1, N+1):
    h_n = h(n)
    F += 2*heston_char(h_n) * exp(-1j*a*h_n) * g_n(n)

  F = exp(-r*T)/(b-a) * np.real(F)
  F = F if opt_type == 'p' else F + S0 - K*exp(-r*T)
  return F if F > 0 else 0




In [1]:
S0      = 100.      # initial asset price
K       = 50.       # strike
r       = 0.03      # risk free rate
T       = 1/365     # time to maturity

v0=0.4173 ; kappa=0.4352 ; theta=0.2982 ; zeta=1.3856 ; rho=-0.0304

In [1]:
import py_vollib_vectorized

price = 0.10 ; S = 95 ; K = 100 ; t = .2 ; r = .2 ; flag = 'c'

def implied_volatility(price, S, K, t, r, flag):
  return py_vollib_vectorized.vectorized_implied_volatility(
    price, S, K, t, r, flag, q=0.0, on_error='ignore', model='black_scholes_merton',return_as='numpy') 


In [2]:
print('Speed Analysis of the implied volatility Function: Per Option')
%timeit implied_volatility(price, S, K, t, r, flag)

Speed Analysis of the implied volatility Function: Per Option
130 µs ± 70.7 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [90]:
from nelson_siegel_svensson.calibrate import calibrate_nss_ols
yield_maturities = np.array([1/12, 2/12, 3/12, 4/12, 6/12, 1, 2, 3, 5, 7, 10, 20, 30])
# yields  = np.array([5.30,5.39,5.50,5.50,5.44,5.11,4.33,3.98,3.70,3.66,3.61,3.98,3.84])
# get the first row of the yield rates
yields = yield_rates.iloc[0].values[1:].astype(np.float64)
curve_fit, status = calibrate_nss_ols(yield_maturities,yields)
# yields

In [91]:
curve_fit, status

(NelsonSiegelSvenssonCurve(beta0=-5.345961485007193, beta1=10.920561543787652, beta2=11.074852298011132, beta3=-4.034323942085258, tau1=43.44535134546507, tau2=3.30260991718184),
   message: Desired error not necessarily achieved due to precision loss.
   success: False
    status: 2
       fun: 0.010378206853388534
         x: [ 4.345e+01  3.303e+00]
       nit: 31
       jac: [ 7.105e-07 -6.778e-05]
  hess_inv: [[ 3.425e+04 -2.641e+02]
             [-2.641e+02  2.338e+01]]
      nfev: 312
      njev: 100)

In [5]:
print('Speed Analysis of Fourier_Heston_Put: Per Option')
%timeit Fourier_Heston_Put(S0,K, T, r, kappa, theta, rho, zeta, v0, 'p', N = 1_012, z = 24)

Speed Analysis of Fourier_Heston_Put: Per Option
437 µs ± 159 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [47]:
import yfinance as yf

In [51]:
ticker = yf.Ticker("AAPL")
# ticker.options
ticker.option_chain("2024-04-19").puts

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,AAPL240419P00005000,2024-03-15 15:59:06+00:00,5.0,0.01,0.00,0.00,0.0,0.0,1.0,0,0.500005,False,REGULAR,USD
1,AAPL240419P00010000,2024-03-21 16:05:13+00:00,10.0,0.01,0.00,0.00,0.0,0.0,NaN,0,0.500005,False,REGULAR,USD
2,AAPL240419P00020000,2024-02-20 14:30:01+00:00,20.0,0.01,0.00,0.00,0.0,0.0,NaN,50,0.500005,False,REGULAR,USD
3,AAPL240419P00060000,2024-03-19 19:06:31+00:00,60.0,0.01,0.00,0.00,0.0,0.0,4.0,0,0.500005,False,REGULAR,USD
4,AAPL240419P00065000,2024-02-20 14:43:29+00:00,65.0,0.01,0.00,0.02,0.0,0.0,NaN,35,1.906250,False,REGULAR,USD
5,AAPL240419P00070000,2024-03-22 15:52:50+00:00,70.0,0.01,0.00,0.00,0.0,0.0,4.0,0,0.500005,False,REGULAR,USD
6,AAPL240419P00075000,2024-03-15 19:41:22+00:00,75.0,0.01,0.00,0.00,0.0,0.0,1.0,0,0.500005,False,REGULAR,USD
7,AAPL240419P00080000,2024-04-08 15:08:09+00:00,80.0,0.01,0.00,0.00,0.0,0.0,1.0,0,0.500005,False,REGULAR,USD
8,AAPL240419P00085000,2024-03-05 19:31:38+00:00,85.0,0.02,0.00,0.02,0.0,0.0,5.0,1054,1.375003,False,REGULAR,USD
9,AAPL240419P00090000,2024-03-21 15:14:20+00:00,90.0,0.01,0.00,0.00,0.0,0.0,4.0,0,0.500005,False,REGULAR,USD
